In [1]:
# from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from glob import glob

In [2]:
# 종목 뉴스리스트 페이지별로 전체 html코드를 긁어온 후, 리스트 형식으로 반환
def getData(codeList, newsPage):
    pageList = []
    for code in codeList:
        for page in range(newsPage, newsPage+1):
            newsListPageUrl = 'https://finance.naver.com/item/news_news.naver?code='+str(code)+'&page='+str(page)+'&sm=title_entity_id.basic&clusterId='
            data = requests.get(newsListPageUrl)
            soup = BeautifulSoup(data.content.decode('euc-kr', 'replace'), 'html.parser')
            dataFilter = soup.find('tbody')
            pageList.append(dataFilter)
    return pageList

In [3]:
# 뉴스페이지별로 전체 html코드를 가공하여 각 뉴스기사들의 url주소를 만들어서, 리스트 형식으로 반환
def change(bs4Data, num):
    makeUrlList = []
    newsUrlList = []
    for bs4 in bs4Data:
        change = bs4.select('tbody > tr > td > a')
        for i in range(len(change)):
            if num <= 9:
                newsUrlList.append('https://finance.naver.com'+str(change[i])[21:65]+str(change[i])[69:83]+str(change[i])[87:99]+str(change[i])[103:110]+str(change[i])[114:138])
            elif num <= 99:
                newsUrlList.append('https://finance.naver.com'+str(change[i])[21:65]+str(change[i])[69:83]+str(change[i])[87:99]+str(change[i])[103:111]+str(change[i])[115:139])
            elif num <= 999:
                newsUrlList.append('https://finance.naver.com'+str(change[i])[21:65]+str(change[i])[69:83]+str(change[i])[87:99]+str(change[i])[103:112]+str(change[i])[116:140])
    return newsUrlList

In [4]:
def getNewsData(urlLists):
    newsDateList = []
    newsTitleList = []
    newsTextList = []
    for i in range(len(urlLists)):
        html = requests.get(urlLists[i])
        soup = BeautifulSoup(html.content.decode('euc-kr', 'replace'), 'html.parser')
        newsDateList.append(soup.find('span',{'class':'tah p11'}).get_text())
        newsTitleList.append(soup.find('strong', {'class':'c p15'}).get_text())
        newsText = soup.find('div', {'class':'scr01'})
        if newsText.find('div', {'class':'link_news'}) != None:
            newsText.find('div', {'class':'link_news'}).decompose()
        newsTextList.append(newsText.get_text()[1:-1])
    newsDf = pd.DataFrame({
        '날짜':newsDateList
        , '제목':newsTitleList
        , '내용':newsTextList
    })
    return newsDf

In [5]:
def filescan():
    fileDataList = []
    fileList = glob('./newsdata/*.csv')
    for i in range(len(fileList)):
        fileDataList.append(pd.read_csv('./newsdata/'+fileList[i][11:], encoding='utf-8'))
    hapData = pd.concat(fileDataList)
    hapData.drop_duplicates(['제목'], inplace=True)
    hapData.sort_values(['날짜'], inplace=True)
    hapData.reset_index(inplace=True)
    hapData.drop('index', axis=1, inplace=True)
    hapData.to_csv('./newsdata/루닛 뉴스 데이터.csv', encoding='utf-8', index=False)

In [6]:
def openFile():
    filtering = pd.read_csv('./newsdata/루닛 뉴스 데이터.csv', encoding='utf-8')
    for i in range(0, len(filtering)):
        filtering['날짜'][i] = filtering['날짜'][i].strip()
        filtering['내용'][i] = str(filtering['내용'][i]).replace('\t','')
    filtering.to_csv('./newsdata/루닛 뉴스 데이터.csv', encoding='utf-8', index=False)
    return pd.read_csv('./newsdata/루닛 뉴스 데이터.csv', encoding='utf-8')

In [9]:
stockCodeList = ['328130'] # 원하는 종목
stockNewsPage = 45         # 원하는 뉴스페이지 숫자 1 ~ 999페이지까지만 긁어오기 가능. # 원하는 페이지수까지에 1을 더해서 입력해야한다.

for numberCount in range(33, stockNewsPage):
    bs4Data = getData(stockCodeList, numberCount)
    urlList = change(bs4Data, numberCount)
    getNewsDf = getNewsData(urlList)
    getNewsDf.to_csv('./newsdata/루닛 뉴스 데이터'+str(numberCount)+'.csv', encoding='utf-8', index=False) # 원하는 종목 코드에 맞는 종목 이름으로 꼭 바꿔서 사용하세요.

# 진행도는 여기서 확인할 수 없으니, ./newsdata/안에 있는 파일 생성되는 것을 확인하세요. -> 네이버 기사 페이지 숫자가 파일명에 붙어서 만들어집니다.
# 예) 하이닉스 뉴스 데이터77.csv -> 하이닉스 뉴스 77페이지 뉴스 리스트 다 긁어옴
# 네이버 봇이 자동으로 막는 문제인지 알 수 없으나, 30~50개 정도 파일을 만들어낼때마다 'Attribute error'가 뜨는데
# 하이닉스 뉴스 데이터93.csv 까지 파일이 만들어졌다가 에러가 떳다면
# for numberCount in range(93, stockNewsPage) <- 이 코드에서 네이버 뉴스 페이지 수 93을 
# 입력한채로 다시 실행하면 다시 만들기가 시작됩니다.
# 에러가 안뜰 수도 있음.
# 테스트 결과 하이닉스 총 344페이지 뉴스 페이지 전부 가져와지는 것을 확인함.

filescan()
openFile()

,날짜,제목,내용
0,2022.07.19 09:34,"AI 진단기업 루닛 21일 상장, 투자자들 유의할 점은",피어그룹 AI 진단 상장사 제외상장 직후 유통 가능 물량 49%2025년 순이익 5...
1,2022.07.19 19:06,"거래소 ""루닛 신규 상장 승인""…기술특례로 코스닥 입성",서울 여의도 한국거래소 전경. 2015.7.2/뉴스1 © News1 박세연 기자(서...
2,2022.07.20 16:14,"루닛·에이프릴, 흥행실패에 조달규모도 축소…어떤 투자 줄였나","[이달 코스닥 상장 앞둬공모가 희망 최하단보다 20~30% ↓]루닛, 에이프릴바이..."
3,2022.07.21 09:09,"[특징주]루닛, 시초가 공모가 소폭 웃돌아…10%대 상승중",[이데일리 안혜신 기자] 루닛(328130)이 공모가를 소폭 웃도는 수준에서 시초...
4,2022.07.21 09:13,"[특징주] 루닛, 코스닥 상장 첫날 ‘상승’",루닛 주가가 코스닥 입성 첫날 급등하고 있다.21일 오전 9시 11...
...,...,...,...
401,2023.07.05 14:12,3만원→18만원된 루닛…초기 투자자 팔았는데도 주가 ‘고공행진’,루닛 2대 주주 웰어라이크 60만주 장내매도향후 초기 투자자 등 차익 실현 가능성 ...
402,2023.07.05 14:55,"루닛, 사우디 가상병원 프로젝트 참여…'AI 암 검진’ 솔루션 제공",빈 살만 사우디 왕자 ‘SEHA 가상병원’ 프로젝트 루닛의 흉부·유방촬영술 AI ...
403,2023.07.05 16:34,"[주가동향] ""187에 탑승했는데 잘한건가요"" 루닛 주가 고점 앞두고 개미 의견 분분",루닛 주가 20만원 임박고점 앞두고 개미 의견 분분언론보도 힘입은 급등 패턴개인투자...
404,2023.07.09 18:48,"큰손들, 금양·루닛·현대로템도 담았네",마켓 PRO 부자들의 추천종목대형 증권사 고액 자산가 고객들이 삼성전자와 포스코홀딩...
